In [1]:
#dependencies and setup
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
# from pprint import pprint
# import os
# import re
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\mosab\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


In [4]:
names = []
categories = []
risks = []
stds = []
srs = []
betas = []
r2s = []

for i in range(1,98):
    fidelity_url = f"https://fundresearch.fidelity.com/fund-screener/results/table/risk/averageAnnualReturnsYear3/desc/{i}?assetClass=&category=&order=assetClass%2Ccategory"
    browser.visit(fidelity_url)
    time.sleep(4)

    # create HTML object
    html = browser.html

    # parse HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    div = soup.find('div', id ='static-table-container')
    table = div.find('table', id = 'static-table')
    tbody = table.find('tbody', id = 'static-tbody')
    for listing in tbody.find_all('td', class_ = 'name left'):
        for name in listing.find_all('a'):
            names.append(name.text)

    div2 = soup.find('div', id ='scrollable-results-table-wrapper')
    table2 = div2.find('table', id = 'scrollable-results-table')
    tbody2 = table2.find('tbody', id = 'results-tbody')
    for listing in tbody2.find_all('td', class_ = "morningstarCategory left"):
        category = listing.find('span').text
        categories.append(category)
    for listing in tbody2.find_all('td', class_ = "morningstarCategoryRisk center"):
        risk = listing.find('div', class_ = "risk-icon-gradient")
        risks.append(risk.get("class", "")[2][-1])
    for listing in tbody2.find_all('td', class_ = "standardDeviation right"):
        if type(listing.find('span')) == type(None):
            stds.append("")
        else:
            stds.append(listing.find('span').text)
    for listing in tbody2.find_all('td', class_ = "sharpeRatio3Yr right"):
        if type(listing.find('span')) == type(None):
            srs.append("")
        else:
            srs.append(listing.find('span').text)
    for listing in tbody2.find_all('td', class_ = "beta right"):
        if type(listing.find('span')) == type(None):
            betas.append("")
        else:
            betas.append(listing.find('span').text)
    for listing in tbody2.find_all('td', class_ = "r2 right"):
        if type(listing.find('span')) == type(None):
            r2s.append("")
        else:
            r2s.append(listing.find('span').text)
    print(f"{len(names)} funds scraped until page {i}")

100 funds scraped until page 1
200 funds scraped until page 2
300 funds scraped until page 3
400 funds scraped until page 4
500 funds scraped until page 5
600 funds scraped until page 6
700 funds scraped until page 7
800 funds scraped until page 8
900 funds scraped until page 9
1000 funds scraped until page 10
1100 funds scraped until page 11
1200 funds scraped until page 12
1300 funds scraped until page 13
1400 funds scraped until page 14
1500 funds scraped until page 15
1600 funds scraped until page 16
1700 funds scraped until page 17
1800 funds scraped until page 18
1900 funds scraped until page 19
2000 funds scraped until page 20
2100 funds scraped until page 21
2200 funds scraped until page 22
2300 funds scraped until page 23
2400 funds scraped until page 24
2500 funds scraped until page 25
2600 funds scraped until page 26
2700 funds scraped until page 27
2800 funds scraped until page 28
2900 funds scraped until page 29
3000 funds scraped until page 30
3100 funds scraped until pag

In [5]:
df_risk = pd.DataFrame({'name' : names, 'morningstar_category': categories, 'risk': risks, 'std_dev': stds,\
                            'sharpe_ratio_3_yr': srs, 'beta': betas, 'r2': r2s})
df_risk.head()

,name,morningstar_category,risk,std_dev,sharpe_ratio_3_yr,beta,r2
0,Baron Partners Fund Institutional Shares (BPTIX),Large Growth,6,40.41,1.60,1.51,0.63
1,Baron Partners Fund Retail Shares (BPTRX),Large Growth,6,40.39,1.59,1.51,0.63
2,"Morgan Stanley Institutional Fund, Inc. Incept...",Small Growth,7,40.44,1.34,1.40,0.71
3,"Morgan Stanley Institutional Fund, Inc. Incept...",Small Growth,7,40.48,1.32,1.40,0.71
4,"Morgan Stanley Institutional Fund, Inc. Incept...",Small Growth,7,40.42,1.30,1.40,0.71


In [6]:
df_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9626 entries, 0 to 9625
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  9626 non-null   object
 1   morningstar_category  9626 non-null   object
 2   risk                  9626 non-null   object
 3   std_dev               9626 non-null   object
 4   sharpe_ratio_3_yr     9626 non-null   object
 5   beta                  9626 non-null   object
 6   r2                    9626 non-null   object
dtypes: object(7)
memory usage: 526.5+ KB


In [7]:
# close the browser session    
browser.quit()

In [9]:
df_risk.to_csv('fidelity_mutual_fund_risk.csv', index = False)